<a href="https://colab.research.google.com/github/Palaeoprot/PalaeoProtCourse/blob/main/Tartu_Excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paleoproteomics Data Analysis Workflow

This notebook is designed for the Paleoproteomics Course (March 17-21, 2025) to help you understand how to process and analyze mass spectrometry data from NovorCloud. Each section is explained to help you understand the purpose of the code and how it works.


## Mount Google Drive
This section mounts your Google Drive to access files stored there. This is necessary because we'll be working with data files from NovorCloud that are stored in Google Drive.


In [1]:
# Mount Google Drive (if using Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setting Up the Environment
This section imports all the required libraries for our analysis. We'll use several Python libraries:
- Standard libraries for file handling, regular expressions, and data structures
- NumPy and Pandas for data manipulation
- Matplotlib and Seaborn for data visualization

In [31]:
# Standard library imports
import os
import re
import json
import zipfile
from collections import Counter
from glob import glob

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## File Extraction and Organization
This section identifies and extracts data from zip files that were downloaded from NovorCloud. The code:
1. Looks for all zip files in the specified directory
2. Creates an output directory for extracted files if it doesn't exist
3. Extracts two types of files: parameters.js (search settings) and .peps.txt (peptide identifications)
4. Parses the filenames to extract metadata (student, sample, conditions)
5. Creates dataframes to track the extracted files and their metadata
6. Saves this metadata to CSV files for reference

In [ ]:
# Set the paths
base_path = '/content/drive/MyDrive/2_Teaching/4 External Courses/Tartu Proteomics Course/Wednesday/NovorCloud Output'
extracted_dir = '/content/drive/MyDrive/2_Teaching/4 External Courses/Tartu Proteomics Course/Wednesday/Extracted_Files'

# Create output directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Process all zip files in the directory
zip_files = glob.glob(os.path.join(base_path, '*.zip'))
print(f"Found {len(zip_files)} zip files")

# Track extracted files
extracted_peps_files = []
extracted_params_files = []

# Process each zip file
for zip_path in zip_files:
    zip_name = os.path.basename(zip_path).split('.')[0]  # Get filename without extension

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # List all files in the zip
        file_list = zip_ref.namelist()

        # Extract parameters.js
        params_files = [f for f in file_list if f.endswith('parameters.js')]
        for params_file in params_files:
            # Create output path with zip name prefix
            output_file = os.path.join(extracted_dir, f"{zip_name}_parameters.js")

            # Extract the file
            with zip_ref.open(params_file) as source, open(output_file, 'wb') as target:
                target.write(source.read())

            extracted_params_files.append(output_file)
            print(f"Extracted: {params_file} to {output_file}")

        # Extract .peps.txt files from subfolders
        peps_files = [f for f in file_list if f.endswith('.peps.txt')]
        for peps_file in peps_files:
            # Get subfolder name from the path
            subfolder = os.path.dirname(peps_file).split('/')[-1]
            # Create descriptive output filename
            peps_base = os.path.basename(peps_file)
            output_file = os.path.join(extracted_dir, f"{zip_name}_{subfolder}_{peps_base}")

            # Extract the file
            with zip_ref.open(peps_file) as source, open(output_file, 'wb') as target:
                target.write(source.read())

            extracted_peps_files.append(output_file)
            print(f"Extracted: {peps_file} to {output_file}")

print(f"\nExtraction complete:")
print(f"- {len(extracted_params_files)} parameters.js files extracted")
print(f"- {len(extracted_peps_files)} .peps.txt files extracted")

# Parse filenames to extract metadata
def parse_extracted_filename(filename):
    """
    Parse the extracted filename to get metadata
    Format: zipname_subfolder_filename.peps.txt or zipname_parameters.js
    """
    base = os.path.basename(filename)
    parts = base.split('_')

    if filename.endswith('parameters.js'):
        # This is a parameters file
        return {
            'zip_name': '_'.join(parts[:-1]),  # Everything before _parameters.js
            'file_type': 'parameters',
            'filename': base
        }
    else:
        # This is a peps file - try to extract student, sample, and conditions
        try:
            # Assuming format: StudentInitials-SampleName-Conditions_subfolder_filename.peps.txt
            zip_parts = parts[0].split('-')
            if len(zip_parts) >= 3:
                return {
                    'student': zip_parts[0],
                    'sample': zip_parts[1],
                    'conditions': zip_parts[2],
                    'subfolder': parts[1] if len(parts) > 2 else '',
                    'file_type': 'peps',
                    'filename': base
                }
            else:
                return {
                    'zip_name': parts[0],
                    'subfolder': parts[1] if len(parts) > 2 else '',
                    'file_type': 'peps',
                    'filename': base
                }
        except Exception as e:
            # If parsing fails, return basic info
            return {
                'zip_name': parts[0],
                'file_type': 'peps' if filename.endswith('.peps.txt') else 'unknown',
                'filename': base,
                'error': str(e)
            }

# Create dataframes to track files and metadata
peps_metadata = [parse_extracted_filename(f) for f in extracted_peps_files]
params_metadata = [parse_extracted_filename(f) for f in extracted_params_files]

peps_df = pd.DataFrame(peps_metadata)
params_df = pd.DataFrame(params_metadata)

# Display summary of extracted files
if not peps_df.empty:
    print("\nPeptide files summary:")
    if 'student' in peps_df.columns and 'sample' in peps_df.columns:
        student_counts = peps_df['student'].value_counts()
        sample_counts = peps_df['sample'].value_counts()
        print(f"Files by student: {dict(student_counts)}")
        print(f"Files by sample: {dict(sample_counts)}")
    else:
        print("Could not extract detailed metadata from filenames")
        print(f"Files by zip source: {dict(peps_df['zip_name'].value_counts())}")

# Save metadata to CSV for reference
peps_df.to_csv(os.path.join(extracted_dir, 'peps_files_metadata.csv'), index=False)
params_df.to_csv(os.path.join(extracted_dir, 'params_files_metadata.csv'), index=False)

## Enriching the Data
This section loads the combined data file and enhances it by processing the various parameters from the search. The code:
1. Loads the previously combined peptide and parameter data
2. Processes parameter columns that contain complex data (like lists of PTMs)
3. Creates boolean columns for each type of parameter (e.g., each enzyme type, each fixed or variable PTM)
4. Extracts raw filename and database filename information
5. Parses actual sample types based on raw filenames
6. Provides detailed information about the updated dataframe
7. Saves the enriched dataframe to a new CSV file


This enrichment makes the data easier to analyze because it transforms complex nested data into simple boolean columns.



In [ ]:
#Load the combined data
file_path = '/content/combined_peps_parameters.csv'
df = pd.read_csv(file_path)

print("Initial DataFrame shape (rows, columns):", df.shape)

# Process parameter columns that are currently stored as strings but represent lists or objects
params_to_process = [
    'spectraFilenames',
    'dbFilenames',
    'fixedPtms',
    'variablePtms'
]

# Extract values from string representations of lists
for param in params_to_process:
    if param in df.columns:
        # Create new columns for each item in these lists
        try:
            # First, try to parse the string representations of lists
            new_values = {}

            for idx, value in df[param].items():
                if pd.isna(value):
                    continue

                # Handle string representations of lists
                if isinstance(value, str):
                    # Clean up the string to make it properly parseable
                    clean_value = value.replace("'", '"')  # Replace single quotes with double quotes

                    try:
                        # Try to parse as JSON
                        parsed = json.loads(clean_value)
                    except:
                        try:
                            # Try to parse as Python literal
                            parsed = ast.literal_eval(value)
                        except:
                            # If all parsing fails, just use the string as is
                            parsed = value

                    new_values[idx] = parsed
                else:
                    new_values[idx] = value

            # Create a new series with the parsed values
            parsed_series = pd.Series(new_values)

            # For list columns, create individual columns for each item
            if len(parsed_series) > 0 and isinstance(parsed_series.iloc[0], list):
                # Get all unique values across all lists
                all_items = set()
                for idx, items in parsed_series.items():
                    if isinstance(items, list):
                        all_items.update(items)

                # Create a column for each item
                for item in sorted(all_items):
                    col_name = f"{param}_{item.replace(' ', '_').replace('(', '').replace(')', '')}"
                    df[col_name] = False

                    # Mark True where the item exists in the list
                    for idx, items in parsed_series.items():
                        if isinstance(items, list) and item in items:
                            df.at[idx, col_name] = True

            # For scalar values, just keep the original column
            else:
                df[param] = parsed_series

        except Exception as e:
            print(f"Error processing {param}: {e}")

# Extract raw filename from spectraFilenames
if 'spectraFilenames' in df.columns:
    def get_first_raw_file(value):
        if pd.isna(value):
            return None

        try:
            if isinstance(value, list) and len(value) > 0:
                return value[0]
            elif isinstance(value, str):
                # Try to parse as a list if it looks like one
                if value.startswith('[') and value.endswith(']'):
                    parsed = ast.literal_eval(value)
                    if isinstance(parsed, list) and len(parsed) > 0:
                        return parsed[0]
                return value
        except:
            pass

        return None

    df['raw_filename'] = df['spectraFilenames'].apply(get_first_raw_file)

# Extract database filename from dbFilenames
if 'dbFilenames' in df.columns:
    def get_first_db_file(value):
        if pd.isna(value):
            return None

        try:
            if isinstance(value, list) and len(value) > 0:
                return value[0]
            elif isinstance(value, str):
                # Try to parse as a list if it looks like one
                if value.startswith('[') and value.endswith(']'):
                    parsed = ast.literal_eval(value)
                    if isinstance(parsed, list) and len(parsed) > 0:
                        return parsed[0]
                return value
        except:
            pass

        return None

    df['db_filename'] = df['dbFilenames'].apply(get_first_db_file)

# Extract enzyme information
if 'enzymeSelection' in df.columns:
    # Create boolean columns for each enzyme type
    unique_enzymes = df['enzymeSelection'].unique()
    for enzyme in unique_enzymes:
        if pd.notna(enzyme):
            col_name = f"enzyme_{enzyme}"
            df[col_name] = df['enzymeSelection'] == enzyme

# Extract PTM information into separate columns
def create_ptm_columns(row):
    """Create boolean columns for each PTM type"""
    result = {}

    # Process fixed PTMs
    if 'fixedPtms' in df.columns:
        fixed_ptms = row['fixedPtms']
        if isinstance(fixed_ptms, list):
            for ptm in fixed_ptms:
                result[f"fixed_ptm_{ptm.replace(' ', '_')}"] = True

    # Process variable PTMs
    if 'variablePtms' in df.columns:
        var_ptms = row['variablePtms']
        if isinstance(var_ptms, list):
            for ptm in var_ptms:
                result[f"var_ptm_{ptm.replace(' ', '_')}"] = True

    return result

# Apply the function and add the columns
if 'fixedPtms' in df.columns or 'variablePtms' in df.columns:
    # First create all possible PTM columns
    all_fixed_ptms = set()
    all_var_ptms = set()

    for idx, row in df.iterrows():
        # Fixed PTMs
        if 'fixedPtms' in df.columns:
            fixed_ptms = row['fixedPtms']
            if isinstance(fixed_ptms, list):
                all_fixed_ptms.update(fixed_ptms)

        # Variable PTMs
        if 'variablePtms' in df.columns:
            var_ptms = row['variablePtms']
            if isinstance(var_ptms, list):
                all_var_ptms.update(var_ptms)

    # Create columns for all PTMs
    for ptm in all_fixed_ptms:
        col_name = f"fixed_ptm_{ptm.replace(' ', '_').replace('(', '').replace(')', '')}"
        df[col_name] = False

    for ptm in all_var_ptms:
        col_name = f"var_ptm_{ptm.replace(' ', '_').replace('(', '').replace(')', '')}"
        df[col_name] = False

    # Now fill in the values
    for idx, row in df.iterrows():
        # Fixed PTMs
        if 'fixedPtms' in df.columns:
            fixed_ptms = row['fixedPtms']
            if isinstance(fixed_ptms, list):
                for ptm in fixed_ptms:
                    col_name = f"fixed_ptm_{ptm.replace(' ', '_').replace('(', '').replace(')', '')}"
                    df.at[idx, col_name] = True

        # Variable PTMs
        if 'variablePtms' in df.columns:
            var_ptms = row['variablePtms']
            if isinstance(var_ptms, list):
                for ptm in var_ptms:
                    col_name = f"var_ptm_{ptm.replace(' ', '_').replace('(', '').replace(')', '')}"
                    df.at[idx, col_name] = True

# Extract sample from raw filename
def extract_sample_from_raw(raw_filename):
    if pd.isna(raw_filename):
        return "unknown"

    raw_str = str(raw_filename).lower()

    if 'bovine' in raw_str or 'cow' in raw_str:
        return "cow"
    elif 'mammoth' in raw_str:
        return "mammoth"
    elif 'edmontosaur' in raw_str:
        return "edmontosaurus"
    elif 'turkey' in raw_str:
        return "turkey"
    else:
        return raw_str.split('.')[0]

# Add actual_sample column based on raw filename
if 'raw_filename' in df.columns:
    df['actual_sample'] = df['raw_filename'].apply(extract_sample_from_raw)

# Print information about the updated dataframe
print("\nUpdated DataFrame shape (rows, columns):", df.shape)
print("\nNew columns added:")
new_cols = set(df.columns) - set(['line', 'target/decoy', 'nDecoy', 'specId', 'scanNum', 'mz', 'z', 'ppm', 'score',
                             'peptide', 'protein', 'student', 'sample', 'conditions', 'original_filename', 'notes',
                             'jobTitle', 'spectraFilenames', 'dbFilenames', 'fixedPtms', 'variablePtms',
                             'enzymeSelection', 'fragmentationSelection', 'massAnalyzer', 'precursorErrorTol',
                             'precursorErrorTolUnit', 'fragmentErrorTol', 'fragmentErrorTolUnit',
                             'activateBuiltInPtms', 'peps_file', 'param_file', 'condition'])
for col in sorted(new_cols):
    print(f"  {col}")

# Show sample distribution
if 'actual_sample' in df.columns:
    print("\nSample distribution:")
    sample_counts = df['actual_sample'].value_counts()
    for sample, count in sample_counts.items():
        print(f"  {sample}: {count} entries")

# Find unique combinations of samples and methods
if 'actual_sample' in df.columns:
    # Use database file as a proxy for search strategy
    if 'db_filename' in df.columns:
        print("\nUnique sample-database combinations:")
        combinations = df.groupby(['actual_sample', 'db_filename']).size().reset_index(name='count')
        for _, row in combinations.iterrows():
            print(f"  {row['actual_sample']} with {row['db_filename']}: {row['count']} entries")

        # Find samples analyzed with multiple databases
        sample_db_counts = df.groupby('actual_sample')['db_filename'].nunique()
        samples_with_multiple_dbs = sample_db_counts[sample_db_counts > 1].index.tolist()

        print(f"\nSamples analyzed with multiple databases: {samples_with_multiple_dbs}")

        # For each such sample, show the databases
        for sample in samples_with_multiple_dbs:
            dbs = sorted(df[df['actual_sample'] == sample]['db_filename'].unique())
            print(f"  {sample}:")
            for db in dbs:
                count = len(df[(df['actual_sample'] == sample) & (df['db_filename'] == db)])
                print(f"    {db}: {count} entries")

        # Now, for each sample with multiple databases, compare the spectrum identifications
        for sample in samples_with_multiple_dbs:
            print(f"\n===== COMPARING DATABASES FOR {sample.upper()} =====")

            # Get data for this sample
            sample_data = df[df['actual_sample'] == sample]

            # Get unique databases for this sample
            dbs = sorted(sample_data['db_filename'].unique())

            # Get spectra identified under each database
            db_spectra = {}
            for db in dbs:
                db_df = sample_data[sample_data['db_filename'] == db]
                spectra = set(db_df['specId'].astype(str))
                db_spectra[db] = spectra
                print(f"  {db}: {len(spectra)} spectra identified")

            # Compare first two databases (for simplicity)
            if len(dbs) >= 2:
                db1, db2 = dbs[0], dbs[1]

                # Calculate overlap
                overlap = db_spectra[db1].intersection(db_spectra[db2])
                only_db1 = db_spectra[db1] - db_spectra[db2]
                only_db2 = db_spectra[db2] - db_spectra[db1]

                # Calculate overlap percentages
                overlap_pct_of_db1 = len(overlap) / len(db_spectra[db1]) * 100 if db_spectra[db1] else 0
                overlap_pct_of_db2 = len(overlap) / len(db_spectra[db2]) * 100 if db_spectra[db2] else 0

                print(f"\n  {db1} vs {db2}:")
                print(f"    Overlap: {len(overlap)} spectra ({overlap_pct_of_db1:.1f}% of {db1}, {overlap_pct_of_db2:.1f}% of {db2})")
                print(f"    Only in {db1}: {len(only_db1)} spectra")
                print(f"    Only in {db2}: {len(only_db2)} spectra")

# Save the updated DataFrame
df.to_csv('/content/enriched_peps_parameters.csv', index=False)
print("\nEnriched DataFrame saved to: /content/enriched_peps_parameters.csv")


## Data Exploration
This section provides a comprehensive overview of the dataframe structure and contents. The code:
1. Shows the overall shape of the dataframe (number of rows and columns)
2. Lists all column names with their indices for easy reference
3. Displays the data types of each column
4. Shows the first 5 rows of the dataframe

This helps you understand what data you're working with and how it's structured before you begin your analysis.

In [ ]:
# Display basic information about the dataframe
print("DataFrame shape (rows, columns):", df.shape)

# Display column names
print("\nColumn names:")
for i, col in enumerate(df.columns):
    print(f"{i+1}. {col}")

# Display data types of each column
print("\nData types:")
print(df.dtypes)

# First 5 rows
print("\nFirst 5 rows:")
print(df.head())


## Next Steps for Analysis
Now that the data is prepared and explored, you can add additional analysis cells to:

1. **Filter peptides by sample type**:

In [34]:
   # Example: Select only peptides from the mammoth sample
   mammoth_peptides = df[df['actual_sample'] == 'mammoth']
   print(f"Found {len(mammoth_peptides)} mammoth peptides")

Found 50457 mammoth peptides


2. **Analyze peptide modifications**:

In [36]:
# Example: Count peptides with hydroxyproline modifications
hyp_cols = [col for col in df.columns if 'hydroxy' in col.lower() and col.startswith('variable')]
has_hyp = df[hyp_cols].any(axis=1)
print(f"Found {has_hyp.sum()} peptides with hydroxyproline modifications")

Found 49661 peptides with hydroxyproline modifications


3. **Compare identification efficiency across samples**:

In [ ]:
# Example: Compare how many unique peptides were identified per sample
peptides_by_sample = df.groupby('actual_sample')['peptide'].nunique()
print("Unique peptides by sample:")
print(peptides_by_sample)

4. **Visualize score distributions**

In [ ]:
# Example: Create a histogram of peptide scores by sample
plt.figure(figsize=(10, 6))
for sample in df['actual_sample'].unique():
  sample_data = df[df['actual_sample'] == sample]
  plt.hist(sample_data['score'], alpha=0.5, label=sample, bins=20)
plt.legend()
plt.title('Distribution of Peptide Scores by Sample')
plt.xlabel('Score')
plt.ylabel('Count')
plt.show()
